In [0]:
!pip install azure.identity

Requirement already satisfied: azure.identity in /databricks/python3/lib/python3.8/site-packages (1.7.0)
Requirement already satisfied: msal<2.0.0,>=1.12.0 in /databricks/python3/lib/python3.8/site-packages (from azure.identity) (1.15.0)
Requirement already satisfied: six>=1.12.0 in /databricks/python3/lib/python3.8/site-packages (from azure.identity) (1.15.0)
Requirement already satisfied: cryptography>=2.5 in /databricks/python3/lib/python3.8/site-packages (from azure.identity) (3.4.7)
Requirement already satisfied: msal-extensions~=0.3.0 in /databricks/python3/lib/python3.8/site-packages (from azure.identity) (0.3.0)
Requirement already satisfied: azure-core<2.0.0,>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure.identity) (1.19.0)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.11.0->azure.identity) (2.25.1)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.5->azure.identity) (1.14.5)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi>=1.12->cryptography>=2.5->azure.identity) (2.20)
Requirement already satisfied: PyJWT[crypto]<3,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from msal<2.0.0,>=1.12.0->azure.identity) (2.3.0)
Requirement already satisfied: portalocker~=1.0 in /databricks/python3/lib/python3.8/site-packages (from msal-extensions~=0.3.0->azure.identity) (1.7.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.11.0->azure.identity) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.11.0->azure.identity) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.11.0->azure.identity) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.11.0->azure.identity) (2.10)
 WARNING: You are using pip version 21.0.1; however, version 21.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
# Getting the authentication secrets from Databricks scope
client_id = dbutils.secrets.get(scope="mlops",key="client_id")
client_secret = dbutils.secrets.get(scope="mlops",key="client_secret")
tenant_id = dbutils.secrets.get(scope="mlops",key="tenant_id")
scope = dbutils.secrets.get(scope="mlops",key="scope")

In [0]:
from azure.identity import ClientSecretCredential, InteractiveBrowserCredential, UsernamePasswordCredential

# Function to get access token by providing authentication secrets
def get_access_token(client_id,client_secret,tenant_id,scope="https://analysis.windows.net/powerbi/api/.default"):
    client_secret_credential_class = ClientSecretCredential(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret)
    access_token_class = client_secret_credential_class.get_token(scope)
    access_token_string = access_token_class.token
    print("Access Token Created....")
    return access_token_string

In [0]:
import json
import requests
from datetime import datetime

# Function to refresh dataset by providing access token along with workspace and dataset name
def refresh_dataset(access_token,workspace_name,dataset_name):
  header = {'Authorization': f'Bearer {access_token}'}
  try :
      workspace_request_url = 'https://api.powerbi.com/v1.0/myorg/groups'
      workspaces_request_response = json.loads(requests.get(url=request_url, headers=header).content)
  except Exception as error :
      print("Request Error : ",error)
      return []
  try :
      workspace_id = [workspace_content['id'] for workspace_content in workspaces_request_response['value'] if workspace_content['name'] == workspace_name][0]
  except:
      print("No workspace with workspace name : {} exists...".format(workspace_name))
      return []
  try :  
      datasets_request_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspace_id + '/datasets'
      datasets_request_response = json.loads(requests.get(url=datasets_request_url, headers=header).content)
  except Exception as error:
      print("Request Error : ",error)
      return []
  try :
      dataset_id = [dataset_content['id'] for dataset_content in datasets_request_response['value'] if dataset_content['name'] == dataset_name][0]
  except Exception as error : 
      print("No dataset with dataset name : {} exists in workspace : {}.....".format(dataset_name,workspace_name))
      return []
  try : 
      refresh_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspace_id + '/datasets/' + dataset_id + '/refreshes'
      requests.post(url=refresh_url, headers=header)
      current_time = datetime.now().strftime("%H:%M:%S")
      print("Dataset with Dataset Name : {} in Workspace : {} Refreshed at Time : {}".format(dataset_name,workspace_name,current_time))
  except Exception as error :
      print("Request Error : ",error)
      return []
  return [(workspace_name,workspace_id),(dataset_name,dataset_id)]

In [0]:
# Driver Program
workspace_name = 'Trigger Workspace'
dataset_name = 'Table'

access_token = get_access_token(client_id,client_secret,tenant_id)
refresh_dataset(access_token,workspace_name,dataset_name)

Access Token Created....
Dataset with Dataset Name : Table in Workspace : Trigger Workspace Refreshed at Time : 09:51:30
Out[46]: [('Trigger Workspace', 'b945f1fd-fc0f-4dbd-911c-faad4a89028f'),
 ('Table', '32d9204d-89b7-4b1d-8f48-0ba907367dd4')]